# 🚀 Advanced MLflow: Hyperparameter Tuning, Registry, and Inference

Welcome to the next step in our MLOps journey! While our previous script showed how to track a single model, real-world machine learning involves testing dozens of combinations and carefully managing the deployment of the best ones. This notebook introduces automated tuning, the **MLflow Model Registry**, and model serving.


### 🔍 What this notebook does:

1. **Automated Hyperparameter Tuning:** This script uses the Optuna library to automatically test 30 different parameter combinations (like max depth and number of estimators) for a Random Forest model. 
2. **Nested Tracking Runs:** To keep our MLflow UI tidy, the script uses `nested=True` to group all 30 individual trial runs inside one main "parent" experiment run.
3. **The Model Registry:** Once the tuning is complete, it takes the best-performing model and officially registers it under the name `housing-price-predictor`. 
4. **Promoting to "Champion":** It assigns the alias "champion" to this top model, establishing it as the current best version ready for use.


5. **Load the Model for Predictions (Inference):** It demonstrates how to easily load the "champion" model back into memory using `mlflow.pyfunc.load_model` to predict house prices on new data.
6. **Deploy MLflow Model as a Local Inference Server:** Finally, it shows how to send a JSON data payload to a locally hosted MLflow local inference server to get predictions via HTTP requests (i.e., online inference).

### 🛠️ Instructions for Students:
Run the cells sequentially. Pay close attention to the `objective` function to see how MLflow logs metrics for *each* trial, and notice how the Model Registry allows us to pull our top model back out by its "champion" name without needing to know its exact file path!

In [1]:
import ssl
import mlflow
import optuna
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [2]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
# The set_experiment API creates a new experiment if it doesn't exist.
mlflow.set_experiment("Hyperparameter Tuning Experiment")

2026/02/21 14:20:02 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/02/21 14:20:02 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/02/21 14:20:02 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/02/21 14:20:02 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/02/21 14:20:02 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/02/21 14:20:02 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/02/21 14:20:02 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/21 14:20:02 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/02/21 14:20:02 INFO mlflow.tracking.fluent: Experiment with name 'Hyperparameter Tuning Experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/dasunathukolage/Documents/ITS 2140 (GDSE 69:70)/ml_workshop_1/mlruns/3', creation_time=1771663802701, experiment_id='3', last_update_time=1771663802701, lifecycle_stage='active', name='Hyperparameter Tuning Experiment', tags={}>

In [3]:
X, y = fetch_california_housing(return_X_y=True)
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0)

print(X)
print(X.shape)

[[   8.3252       41.            6.98412698 ...    2.55555556
    37.88       -122.23      ]
 [   8.3014       21.            6.23813708 ...    2.10984183
    37.86       -122.22      ]
 [   7.2574       52.            8.28813559 ...    2.80225989
    37.85       -122.24      ]
 ...
 [   1.7          17.            5.20554273 ...    2.3256351
    39.43       -121.22      ]
 [   1.8672       18.            5.32951289 ...    2.12320917
    39.43       -121.32      ]
 [   2.3886       16.            5.25471698 ...    2.61698113
    39.37       -121.24      ]]
(20640, 8)


In [4]:
def objective(trial):
    # Setting nested=True will create a child run under the parent run.
    with mlflow.start_run(nested=True, run_name=f"trial_{trial.number}") as child_run:
        rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32)
        rf_n_estimators = trial.suggest_int("rf_n_estimators", 50, 300, step=10)
        rf_max_features = trial.suggest_float("rf_max_features", 0.2, 1.0)
        params = {
            "max_depth": rf_max_depth,
            "n_estimators": rf_n_estimators,
            "max_features": rf_max_features,
        }
        # Log current trial's parameters
        mlflow.log_params(params)

        regressor_obj = sklearn.ensemble.RandomForestRegressor(**params)
        regressor_obj.fit(X_train, y_train)

        y_pred = regressor_obj.predict(X_val)
        error = sklearn.metrics.mean_squared_error(y_val, y_pred)
        # Log current trial's error metric
        mlflow.log_metrics({"error": error})

        # Log the model file
        mlflow.sklearn.log_model(regressor_obj, name="model")
        # Make it easy to retrieve the best-performing child run later
        trial.set_user_attr("run_id", child_run.info.run_id)
        return error

In [5]:
# Create a parent run that contains all child runs for different trials
with mlflow.start_run(run_name="study") as run:
    # Log the experiment settings
    n_trials = 30
    mlflow.log_param("n_trials", n_trials)

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=n_trials)

    # Log the best trial and its run ID
    mlflow.log_params(study.best_trial.params)
    mlflow.log_metrics({"best_error": study.best_value})
    if best_run_id := study.best_trial.user_attrs.get("run_id"):
        mlflow.log_param("best_child_run_id", best_run_id)

[I 2026-02-21 14:20:10,755] A new study created in memory with name: no-name-88bffe1a-4a42-4a30-958f-266498349661
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
[I 2026-02-21 14:20:37,289] Trial 0 finished with value: 0.26175076575620193 and parameters: {'rf_max_depth': 24, 'rf_n_estimators': 250, 'rf_max_features': 0.8620054461008557}. Best is trial 0 with value: 0.26175076575620193.
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format 

In [ ]:
# Register the best model using the model URI
mlflow.register_model(
    model_uri="runs:/0f9534135a5e41f2b5448d390c8d04f4/model",
    name="housing-price-predictor",
)

# > Successfully registered model 'housing-price-predictor'.+
# > Created version '1' of model 'housing-price-predictor'.

Successfully registered model 'housing-price-predictor'.
2026/02/21 14:51:48 WARNING mlflow.tracking._model_registry.fluent: Run with id 0f9534135a5e41f2b5448d390c8d04f4 has no artifacts at artifact path 'model', registering model based on models:/m-aefdd165d8d14a6dac2ee3382f058366 instead
Created version '1' of model 'housing-price-predictor'.


<ModelVersion: aliases=[], creation_timestamp=1771665708035, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1771665708035, metrics=None, model_id=None, name='housing-price-predictor', params=None, run_id='0f9534135a5e41f2b5448d390c8d04f4', run_link=None, source='models:/m-aefdd165d8d14a6dac2ee3382f058366', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [ ]:
# Now we can promote the best model to "champion" status by assigning it an alias. This allows us to easily reference the champion model in future code or deployments.
from mlflow import MlflowClient

client = MlflowClient()

# Assign the alias "champion" to version 1 of your model
client.set_registered_model_alias(
    name="housing-price-predictor", 
    alias="champion", 
    version="1"
)
print("Model promoted to Champion!")

Model promoted to Champion!


In [ ]:
# Load the Model for Predictions (Inference)

import pandas as pd

# 1. Connect to the local MLflow tracking server (same as training)
mlflow.set_tracking_uri("sqlite:///mlflow.db")

# 2. Pull the current champion model directly from the registry
model_uri = "models:/housing-price-predictor@champion"

print(f"Loading model from: {model_uri}...")

# Load the model as a generic Python Function (pyfunc)
# This is how MLflow loads models for inference, regardless of the training library
loaded_model = mlflow.pyfunc.load_model(model_uri)

# 3. Create new, unseen data (Must match the 8 features exactly!)
# Here is a fake house located somewhere in California
new_data = pd.DataFrame([{
    "MedInc": 5.23,      # Median income (in tens of thousands, so ~$52,300)
    "HouseAge": 25.0,    # House age in years
    "AveRooms": 5.5,     # Average rooms
    "AveBedrms": 1.2,    # Average bedrooms
    "Population": 1200.0,# Block population
    "AveOccup": 3.0,     # Average people per household
    "Latitude": 36.77,   # Latitude (Central CA)
    "Longitude": -119.41 # Longitude (Central CA)
}])

# 4. Make a prediction
print("\nMaking prediction on new data...")
predictions = loaded_model.predict(new_data)

# The prediction is in units of $100,000 based on our dataset. Let's format it nicely.
predicted_value = predictions[0] * 100000
print(f"Predicted House Value: ${predicted_value:,.2f}")

Loading model from: models:/housing-price-predictor@champion...

Making prediction on new data...
Predicted House Value: $219,650.45


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [ ]:
# If you want to send this data to a registered MLflow model via REST API, you can use the following code snippet:

import requests
import json

# 1. Define the API endpoint (MLflow always uses /invocations for predictions)
url = "http://127.0.0.1:5002/invocations"

# 2. Package our dummy data into the JSON format MLflow expects
# "dataframe_records" is the standard way to send row-based data
payload = {
    "dataframe_records": [
        {
            "MedInc": 8.32,
            "HouseAge": 41.0,
            "AveRooms": 6.98,
            "AveBedrms": 1.02,
            "Population": 322.0,
            "AveOccup": 2.55,
            "Latitude": 37.88,
            "Longitude": -122.23
        }
    ]
}

# 3. Define the headers (Tell the server we are sending JSON data)
headers = {'Content-Type': 'application/json'}

# 4. Fire the POST request to the server!
print("Sending request to MLflow server...")
response = requests.post(url, data=json.dumps(payload), headers=headers)

# 5. Read the response
if response.status_code == 200:
    # The server replies with a JSON containing a "predictions" list
    result = response.json()
    predicted_price = result["predictions"][0] * 100000
    print(f"Server responded! Predicted Value: ${predicted_price:,.2f}")
else:
    print(f"Error {response.status_code}: {response.text}")